## Méthode d'exploration et d'analyse

Dans ce notebook, nous allons découvrir des méthodes basiques pour explorer et analyser une série temporelle. L'objectif est de pouvoir préparer correctement les données avant de tenter d'apprendre un modèle de prédiction.

Ce notebook a été construit à partir de la source suivante : [*Practical Time Series Analysis, Prediction with Statistics & Machine Learning*, Aileen Nielsen, 2020](https://www.oreilly.com/library/view/practical-time-series/9781492041641/).

Les méthodes décritent dans ce notebook sont les suivantes :
- nettoyage des données
- sélection de la granularité temporelle
- visualisation par histogramme
- saisonnalité / tendance / résidue des données
- smoothing

Nous commençons par charger quelques bibliothèques que nous allons utiliser dans la suite :


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

plt.rcParams['figure.figsize'] = [15, 8]

### Chargement des données

Pour illuster les différentes méthodes précédentes, nous allons travailler sur un jeu de données *école* dans le domaine des séries temporelles : le nombre de passager de compagnies aériennes.

**Questions**:
1. étudier le fichier `Air_Traffic_Passenger_Statistics.csv`
2. créer un dataframe à partir de ce fichier, en prenant la première ligne comme colonne de ce dataframe
3. sélectionner les deux colonnes nous intéressant, `['Activity Period', 'Passenger Count']`
4. utiliser les dates comme index
5. documenter vous sur la commande `groupby` de Pandas
7. faire un `groupby` en appliquant une fonction d'aggrégation sur les sous groupes, pour obtenir la somme des passagers sur chaque mois de l'année (il faut sommer le nombre de passagers de toutes les compagnies) 
8. afficher les premières lignes du dataframe pour vérifier que tout est bon
9. afficher les données, avec en X les timestamp, et en Y le nombre de passagers

In [ ]:
fileName = 'Air_Traffic_Passenger_Statistics.csv'

data = ...

# selection colonnes
data = ...

# index sur les dates
data['Activity Period'] = ...

# groupby avec sum()
data = ...

data.head()

In [ ]:
# affichage 


Si tout se passe bien, vous devez obtenir une courbe croissante, s'écrasant en 2020, avec des variations saisonnales.

## Nettoyage et granularité

La première chose à vérifier est l'axe des timestamp de cette série temporelle. Nous allons voir qu'il est possible de simplifier les données rien qu'en changeant la granularité de cet axe.

**Questions** : 
1. quelle est la granularité de cette série ?
2. avez vous des timestamp manquants ? un nombre de passager nulle pour une date précise ?
3. la granularité est elle correcte ? une granularité par année serait mieux ?
4. Pour le vérifier :
    1. créer une copie du dataframe `data` (dans une variable `dataYear`)
    2. ajouter une colonne gardant l'année de chaque index
    3. faire un `groupby` avec une fonction d'aggrégation de somme pour obtenir la somme des passagers par année
    4. afficher les données, avec en X les timestamp, et en Y le nombre de passager
5. quelle information perd-t-on en regroupant par année ?



In [ ]:

# dfCopy = df.copy() permet de copier un dataFrame
dataYear = ...

# df.index permet d'accéder à l'index d'un dataframe
# df.col.to_period(X) permet de changer le format d'un timestamp d'une colonne (ou d'un index, via `df.index`), avec en paramètre 'Y' pour garder l'année, 'M' pour garder le mois, ...
dataYear['year'] = ...

# groupby
dataYear = ...

dataYear.head()

In [ ]:
# affichage


Sur la figure précédente, vous pouvez noter que la courbe obtenue est en quelque sorte une version simplifiée de nos données initiales. Simplifiées mais plus lisible, pour par exemple étudier la tendance de nos données. Ici, nous voyons bien que le nombre de passager augmente au fil des ans. 

Simplifier une série temporelle peut être intéressant pour étudier les données. Nous allons revenir sur ce point, après avoir présenté des méthodes de visualisation et puis avoir étudié comment est décomposée une série temporelle.


## Visualisation

La première chose à faire quand on analyse des données est de tenter de les afficher. Nous avons déjà des méthodes pour réaliser cette tâche dans les notebooks précédents. Reprenons nos données avec une granularité par mois (avec la somme de passagers de toutes les compagnies) et allons-y pour un histogramme afin de voir la distribution des valeurs.

In [ ]:
data.hist(bins=100)

L'histogramme précédent, sur nos données de passagers, nous montre qu'il y a beaucoup de valeurs hautes, ce qui est lié à la tendance haussière du nombre de passager. 

Un histogramme nous fournit l'information suivante : le nombre de valeurs qui apparaissent dans chaque sous ensemble défini sur l'axe des X. Dans notre contexte de série temporelle, une information plus intéressante peut être obtenue en étudiant les variations de valeurs. Notre objectif étant de prévoir ces variations, il est intéressant d'afficher sous forme d'histogramme le nombre de hausses et baisses de nos données.

L'idée consiste donc à transformer nos données en calculant la différence des jours 2 à 2, ce qui se réalise très facilement sur une série, via la fonction `diff()`.

### Questions :
1. documenter vous sur la fonction `diff`
2. générer les données de différence des jours 2 à 2
3. afficher l'histogramme de ces données


In [ ]:
# diff
diff =

# affichage d'un histogramme des différences
diff....

Voici comment interpréter notre histogramme:
- chaque cellule définie sur X nous indique le nombre de fois où une telle hausse est apparue
- en X, à gauche du 0, ce sont les baisses, à droite, les hausses. On note qu'il y a plus de hausses que de baisses, ce qui rejoint la tendance haussière du nombre de passagers.

## Saisonnalité, tendance et résidue

Maintenant que nous avons vu que les données étaient propres, et que la granularité mensuelle permettait de garder l'information de saisonnalité, nous pouvons maintenant définir un peu mieux la notion de saisonnalité.

Beaucoup de séries temporelles sont liées à des activités humaines, elles-même liées au rythme des saisons. Le pic des vols étant réalisé pour les départs en vacances, nous observons une notion de période qui se répète tous les ans. En plus de cette période, nous observons aussi une tendance haussière au fil des ans du nombre de passager.

Une telle série temporelle peut ainsi être définie par trois composantes : la saisonnalité, la tendance et le résidu, qui ne peut être expliqué facilement. En mathématique, cela donnerait :

$$
Y[t] = S[t] + T[t] + R[t]
$$
Avec $S[t]$ la fonction de saisonnalité à l'instant $t$, $T[t]$ la tendance et $R[t]$ le résidue.

L'utilité de cette décomposition est malheureusement motivée par le fait suivant : la majorité des modèles de prédictions fonctionne sur des séries temporelles stationnaires, une série dont la moyenne glissante reste stable. Ce qui veut dire que la série ne doit avoir ni de saisonnalité, ni de tendance. Donc pour appliquer ce type de modèle de prédiction, il faut donc déshabiller votre série...pour qu'il ne reste que le résidue... 

**Questions**:
1. Utiliser la fonction `seasonal_decompose(dataframe)` pour obtenir la décomposition précédente sur les données de passagers aériens
- l'objet retourné par cette fonction contient lui-même trois objets nommé `trend`, `seasonal` et `resid`
- ces trois objets sont des séries Pandas
2. Afficher les données initiales, la tendance, la saisonnalité et le résidue obtenues
- la fonction `plt.subplot(411)` permet de créer une figure contenant elle même 4 lignes et une colonne. Le dernier `1` permet de dire que le prochain appelle affichera la première figure (première ligne de la seule colonne)
- `plt.subplot(412)` indique que vous passez sur la deuxième figure, et ainsi de suite

In [ ]:
# decomposition

decomposition = ...

trend = ...
seasonal = ...
residual = ...


In [ ]:
# affichage 

plt.subplot(411)
plt.plot(data, label='original')
plt.legend(loc="best")
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc="best")
plt.subplot(413)
plt.plot(seasonal, label='Seasonal')
plt.legend(loc="best")
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc="best")
plt.tight_layout()

La dernière figure représente donc ce sur quoi les modèles de prédiction peuvent généralement travailler.

## Smoothing

Nous pouvons maintenant revenir sur les méthodes de simplification (_smoothing_) d'une série temporelle. 
L'objectif de ces méthodes est de pouvoir faire apparaitre les _pattern_ des séries temporelles.  

De nombreuses méthodes existent pour _adoucir_ une série temporelle. L'idée est de remplacer les données par une moyenne de ces données réalisées sur une fenêtre d'une certaine taille. En pratique, la génération de la série temporelle simplifiée est réalisée comme suit :
1. on se positionne sur la position x de la série
2. à partir de x, on récupère les _window_ données aux alentours du point x
3. on ajoute la moyenne de ces points dans la série 

L'étape 2 a plusieurs variantes : la sélection des _window_ données peut se faire à gauche du point, ou bien x peut être centré sur la fenêtre. Dans ce dernier cas, attention car des données futures sont utilisées pour calculer la moyenne. Dans un contexte d'apprentissage de modèle pour faire de la prédiction, cela n'est pas possible/acceptable (le modèle appris nous donnera de bon résultats de prédiction, mais ce même modèle utilisé en situation réelle aura des résultats moins bons).

Notez que certaines données initiales de la série temporelle ne pourront pas être utilisées. La première donnée n'a aucun voisin sur sa gauche, et on ne peut donc pas calculer de moyenne sur ce point. Cela est aussi valable pour la fin de la série si vous utilisez une moyenne centrée (ce qui n'est pas le cas pour une moyenne à partir des données passées).

En Pandas, la méthode `pandas.DataFrame.rolling` fournit une implémentations de ces algorithmes.

**Question(s)**:
1. étudier la fonction `pandas.DataFrame.rolling`
2. supprimer les valeurs `Nan` du résidu
3. _smoothingnitiser_ le résidu avec la `window` centrée et gauche
4. afficher le `head` et `tail` des séries obtenus
5. afficher les séries `residu`, `smoothTrail`, `smoothCentered`

In [ ]:
# Questions 1-2-3

# suppression Nan
residual = residual.dropna()

residual.head()

# smoothTrail et smoothCentered
smoothTrail = ...
smoothCentered = ...


Notez qu'après le calcul des moyennes, certaines valeurs sont nulles :

In [ ]:
# Question 4

# affichage head et trail de smoothTrail et smoothCentered

On peut maintenant afficher le résidue _adouci_ :

In [22]:
# Question 5 - affichage des trois courbes sur une même figure, avec une légende


La courbe verte représente la version centrée et la courbe orange la version à gauche :
- la courbe `smoothTrail` commence après le début de la série initiale et finit bien sur sa fin
- la courbe `smoothCentered` commence après le début de la série temporelle initiale (mais moins longtemps après que la version à gauche) et finit avant la fin (et avant la version gauche)
- la version à gauche est un peu en retard au niveau variation de la série initiale (les pics ont lieu un peu après le pic de la série)
- la version centrée a des pics plus en phase sur la série initiale, ce qui est normal vu qu'on utilise des données futures 

## Conclusion

Nous en avons fini avec des méthodes de visualisation / simplification de série temporelle. Vous pourrez maintenant utiliser ces méthodes pour étudier vos propres séries temporelles. 

On peut maintenant continuer à implémenter nos stratégies de trader, en utilisant ces nouvelles connaissances.

Voici une stratégie de tradeur, récupérée sur le web, restant assez simple, et basée sur la moyenne mobile :
- Règle 1 : lorsque le prix est supérieur à la moyenne mobile, allez seulement à l'achat;
- Règle 2 : lorsque le prix est inférieur à la moyenne mobile, allez seulement à la vente.


**Questions**:
1. documenter vous sur les marchés haussier / baissier 
   [https://fr.wikipedia.org/wiki/Tendance_(%C3%A9conomie)]
2. implémenter la stratégie 
3. tester cette stratégie avec plusieurs séries temporelles, en prenant un contexte haussier et baissier.
